In [1]:
import matplotlib.pyplot as plt 
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
import joblib
from keras.metrics import RootMeanSquaredError,mean_squared_error

In [2]:
def pasar_a_datetime(fecha):
    #format_string = '%Y-%m-%d'
    anio,mes,dia= fecha.split('-')
    aux_dia = dia.split(' ')
    if len(aux_dia)>1:dia = aux_dia[0]
    res = datetime(int(anio),int(mes),int(dia))
    return res

In [3]:
def scale_datasets(x_train, x_test):

  """
  Standard Scale test and train data
  Z - Score normalization
  """
  standard_scaler = StandardScaler()
  x_train_scaled = pd.DataFrame(
      standard_scaler.fit_transform(x_train),
      columns=x_train.columns
  )
  x_test_scaled = pd.DataFrame(
      standard_scaler.transform(x_test),
      columns = x_test.columns
  )
  return x_train_scaled, x_test_scaled

# Idea

Dada un intervalo de fecha retornar la prediccion del modelo

Esos intervalos pueden ser aquellos establecidos en encuestas (agrupados mean())

Guardar ambas tablas, calcular score para cada fecha

In [4]:
publicaciones = pd.read_csv('C:/Users/54911/OneDrive/Escritorio/Data Science/Elecciones + IA/modeling/Preparando_datos/publicaciones_politicas.csv')

In [5]:
def filtrar_fechas(datos_publicaciones, inicio, final):
    aux_datos_publicaciones = datos_publicaciones['fecha'].map(lambda x:pasar_a_datetime(x))
    res = datos_publicaciones[(aux_datos_publicaciones>=inicio) & (aux_datos_publicaciones<=final)]
    return res

In [6]:
def separar_fecha(data):
    data['dia'] = ''
    data['mes'] = ''
    data['anio'] = ''
    data['dia'] = data['fecha_mediana'].map(lambda x:x.day)
    data['mes'] = data['fecha_mediana'].map(lambda x:x.month)
    data['anio'] = data['fecha_mediana'].map(lambda x:x.year)

In [7]:
def encoding_Fuente(tabla,fuentes):
    #fuentes_labels = publicaciones['fuente'].unique()
    numbers = [i+1 for i in range(len(fuentes))]
    simple_encoding = dict(zip(fuentes,numbers))
    tabla['Fuente'] = tabla['Fuente'].map(lambda x:simple_encoding[x])

In [19]:
def formato_entrenado(datos_publicaciones_input):
    datos_publicaciones = datos_publicaciones_input.copy()
    datos_publicaciones['fecha']=datos_publicaciones['fecha'].map(lambda x:pasar_a_datetime(x))
    fuentes = datos_publicaciones['fuente'].unique()
    partidos = ['La Libertad Avanza','Union por la Patria']
    #partidos = publicaciones['Partido'].unique()
    #datos_publicaciones['fecha'] = datos_publicaciones['fecha'].map(lambda x:pasar_a_datetime(x))

    #datos_publicaciones['cantidad_likes'].quantile(0.95)
    datos_publicaciones = datos_publicaciones[datos_publicaciones['cantidad_likes']<25000]

    
    agrupado_likes = datos_publicaciones.groupby(['Partido','fuente'])['cantidad_likes'].median()
    fecha_median = datos_publicaciones['fecha'].median()

    df = pd.DataFrame()

    for p in list(partidos):

        if p in agrupado_likes.index:
            likes_partidos = agrupado_likes[p]
            fuentes_partido = likes_partidos.index
            likes_median = []

            for c in fuentes:
                if c in fuentes_partido:
                    likes_median.append(likes_partidos[c])
                else:
                    likes_median.append(0)
        
            df[p] = likes_median
        else:
            df[p] = [0]*len(fuentes)


    df['Fuente'] = fuentes
    df['fecha_mediana'] = fecha_median

    
    separar_fecha(df)
    df.drop(['fecha_mediana','anio'],axis=1,inplace=True)
    encoding_Fuente(df,fuentes)
    return df

In [21]:
def acomodar_prediccion(prediccion,partidos):

    resultados_mean = []
    for i in range(2):
        resultados_mean.append(prediccion[:,i].mean())

    total = sum(resultados_mean)
    resultados_norm = []
    for r in resultados_mean:
        aux = 100*r/total
        resultados_norm.append(aux)

    resultados_norm = [round(x,1) for x in resultados_norm]

    df = pd.DataFrame({'Partido':partidos,'Resultado':resultados_norm})

    return df,resultados_norm

In [22]:
def prediccion_dada_fecha(model,scaler,inicio,final):
    # inicio = datetime(2023,4,1)
    # final = datetime(2023,4,14)
    periodo_fecha = filtrar_fechas(publicaciones,inicio,final)
    periodo_fecha_formato = formato_entrenado(periodo_fecha)

    #periodo_fecha_formato = periodo_fecha_formato.iloc[:,[1,2,0,4,3,5,6,7]]

    periodo_fecha_formato_scaled = scaler.transform(periodo_fecha_formato)
    prediccion_fecha = model.predict(periodo_fecha_formato_scaled,verbose=0)
    partidos = periodo_fecha_formato.columns[:2]
    df,resultados_norm= acomodar_prediccion(prediccion_fecha,partidos)
    return df

In [10]:
scaler = joblib.load('C:/Users/54911/OneDrive/Escritorio/Data Science/Elecciones + IA/modeling/scaler_primeravuelta.gz')
model_primera_vuelta = load_model('C:/Users/54911/OneDrive/Escritorio/Data Science/Elecciones + IA/modeling/modelo_primera_vuelta')

In [12]:
Encuestas_primera_vuelta = pd.read_csv('C:/Users/54911/OneDrive/Escritorio/Data Science/Elecciones + IA/getting_data/encuestas/Encuestas_primera_vuelta.csv')
Encuestas_agrupado_prim = Encuestas_primera_vuelta.groupby(['Inicio','Final']).mean().reset_index()

partidos = Encuestas_agrupado_prim.columns[2:]

In [16]:
score = []
predicciones = np.zeros((5,len(Encuestas_agrupado_prim)))
for i in range(len(Encuestas_agrupado_prim)):
    
    fila = Encuestas_agrupado_prim.iloc[i]
    inicio = fila['Inicio']
    final = fila['Final']
    predicho = prediccion_dada_fecha(model_primera_vuelta,scaler,pasar_a_datetime(inicio),pasar_a_datetime(final))
    valores_pred = predicho['Resultado'].to_numpy(dtype=float)
    esperado = np.array(fila.values[2:],dtype=float)

    score.append(-mean_squared_error(esperado,valores_pred).numpy())

    for k,p in enumerate(partidos):
        # lista_partido = predicciones[p]
        # lista_partido.append(valores_pred[k])
        predicciones[k,i] =  valores_pred[k]


In [ ]:
predicciones_score_df = pd.DataFrame()
predicciones_score_df['Scores'] = score
for i in range(5):
    predicciones_score_df[partidos[i]] = predicciones[i]

predicciones_score_df['Inicio'] = Encuestas_agrupado_prim['Inicio']
predicciones_score_df['Final'] = Encuestas_agrupado_prim['Final']
predicciones_score_df['Inicio']= predicciones_score_df['Inicio'].map(lambda x:pasar_a_datetime(x))
predicciones_score_df['Final']= predicciones_score_df['Final'].map(lambda x:pasar_a_datetime(x))

predicciones_score_df = predicciones_score_df[predicciones_score_df['Inicio']>datetime(2022,12,31)]
predicciones_score_df = predicciones_score_df[predicciones_score_df['Final']>datetime(2022,12,31)]

predicciones_score_df.to_csv('C:/Users/54911/OneDrive/Escritorio/Data Science/Elecciones + IA/dashboard/predicciones_score_df_prim.csv',index=False)

# Ballotaje

In [11]:
scaler = joblib.load('C:/Users/54911/OneDrive/Escritorio/Data Science/Elecciones + IA/modeling/scaler_ballotaje.gz')
model= load_model('C:/Users/54911/OneDrive/Escritorio/Data Science/Elecciones + IA/modeling/modelo_ballotaje')

In [13]:
Encuestas = pd.read_csv('C:/Users/54911/OneDrive/Escritorio/Data Science/Elecciones + IA/getting_data/encuestas/Encuestas_solo_ballotaje.csv')
Encuestas_agrupado_ = Encuestas.groupby(['Inicio','Final']).mean().reset_index()

partidos = Encuestas_agrupado_.columns[2:]

In [23]:
score = []
predicciones = np.zeros((2,len(Encuestas_agrupado_)))
for i in range(len(Encuestas_agrupado_)):
    
    fila = Encuestas_agrupado_.iloc[i]
    inicio = fila['Inicio']
    final = fila['Final']
    predicho = prediccion_dada_fecha(model,scaler,pasar_a_datetime(inicio),pasar_a_datetime(final))
    valores_pred = predicho['Resultado'].to_numpy(dtype=float)
    esperado = np.array(fila.values[2:],dtype=float)

    score.append(-mean_squared_error(esperado,valores_pred).numpy())

    for k,p in enumerate(partidos):
        # lista_partido = predicciones[p]
        # lista_partido.append(valores_pred[k])
        predicciones[k,i] =  valores_pred[k]


In [24]:
predicciones_score_df = pd.DataFrame()
predicciones_score_df['Scores'] = score
for i in range(2):
    predicciones_score_df[partidos[i]] = predicciones[i]

predicciones_score_df['Inicio'] = Encuestas_agrupado_['Inicio']
predicciones_score_df['Final'] = Encuestas_agrupado_['Final']
predicciones_score_df['Inicio']= predicciones_score_df['Inicio'].map(lambda x:pasar_a_datetime(x))
predicciones_score_df['Final']= predicciones_score_df['Final'].map(lambda x:pasar_a_datetime(x))

predicciones_score_df = predicciones_score_df[predicciones_score_df['Inicio']>datetime(2022,12,31)]
predicciones_score_df = predicciones_score_df[predicciones_score_df['Final']>datetime(2022,12,31)]

predicciones_score_df.to_csv('C:/Users/54911/OneDrive/Escritorio/Data Science/Elecciones + IA/dashboard/predicciones_score_ballotaje.csv',index=False)

In [25]:
predicciones_score_df

,Scores,La Libertad Avanza,Union por la Patria,Inicio,Final
0,-96.290000,48.2,51.8,2023-10-23,2023-10-24
1,-140.785000,48.2,51.8,2023-10-23,2023-10-25
2,-47.450000,49.0,51.0,2023-10-23,2023-10-29
3,-38.090000,49.3,50.7,2023-10-25,2023-10-27
4,-17.210000,50.4,49.6,2023-10-25,2023-11-08
5,-45.046250,50.8,49.2,2023-10-26,2023-11-03
6,-34.625000,50.1,49.9,2023-10-28,2023-10-29
7,-22.713889,51.5,48.5,2023-11-01,2023-11-03
8,-39.625000,51.6,48.4,2023-11-01,2023-11-06
9,-13.140000,51.5,48.5,2023-11-02,2023-11-03
